In [1]:
import os

# 获取当前工作目录
current_dir = os.getcwd()
print("当前工作目录：", current_dir)

# 切换到上一层目录
parent_dir = os.path.dirname(current_dir)
os.chdir(parent_dir)
print("切换后的目录：", parent_dir)

当前工作目录： c:\Users\zhaoxs3\Documents\GitHub\NEMESIS\unit_test
切换后的目录： c:\Users\zhaoxs3\Documents\GitHub\NEMESIS


In [2]:
import QuantLib as ql
import pandas as pd
import numpy as np
from devlib.market.curves.pm_curves import PmForwardCurve
from devlib.products.fx import fx_vanilla, fx_digital, fx_range_accrual, fx_knock
from devlib.market.volatility.fx_vol_surface import FxVolSurfaceHK
from devlib.market.volatility.constant_vol_surface import ConstantVolSurface
from devlib.market.curves.fx_curves import FxImpliedAssetCurve
from devlib.market.curves.overnight_index_curves import Sofr
from devlib.utils.fx_utils import get_calendar

In [3]:
today = ql.Date(11,8,2025)
ql.Settings.instance().evaluationDate = today

# pm forward_curve
d_ccy = 'USD'
f_ccy = 'XAU'
mkt_file_path = './unit_test/data/market_data_fivs_20250811.xlsx'
pm_fwd_data = pd.read_excel(mkt_file_path, sheet_name='precious metal')
pm_fwd_data = pm_fwd_data.loc[pm_fwd_data['TYPE'] == f_ccy+d_ccy, ['TENOR', 'SETTLE_DT', 'PX_LAST']]
pm_fwd_data.columns = ['Tenor', 'SettleDate', 'Rate']
pm_fwd_data.loc[pm_fwd_data.loc[:,'Tenor']!='SPOT', 'Rate'] /= 100
spot = pm_fwd_data.loc[pm_fwd_data.loc[:,'Tenor']=='SPOT', 'Rate'].values[0]
calendar = get_calendar(f_ccy, d_ccy, is_with_usd=True)
fx_fwd_crv = PmForwardCurve(today, spot, pm_fwd_data, f_ccy, d_ccy, calendar=calendar, daycount=ql.Actual365Fixed(), data_type='rate')

# d_curve
mkt_file = "./unit_test/data/sofr_curve_data_20250811.xlsx"
swap_mkt_data = pd.read_excel(mkt_file, sheet_name="swap")
fixing_data = pd.read_excel(mkt_file, sheet_name="fixing")
d_crv = Sofr(today, swap_mkt_data=swap_mkt_data, fixing_data=fixing_data)

# f_curve
calendar = get_calendar(f_ccy, d_ccy, is_with_usd=True)
f_crv = FxImpliedAssetCurve(today, d_crv, fx_fwd_crv, calendar, ql.Actual365Fixed())

# vol_surface
vol_file = "./unit_test/data/XAUUSD_voldata_20250811.xlsx"
vol_data = pd.read_excel(vol_file, sheet_name="vol_data")
calendar = get_calendar(f_ccy, d_ccy, is_with_usd=True)
vol_surface = FxVolSurfaceHK(today, vol_data, spot, d_ccy, f_ccy,
                             fx_fwd_crv, d_crv, f_crv, calendar, daycount=ql.Actual365Fixed())

In [4]:
expiry = ql.Date(13,11,2025)
payment_date = calendar.advance(expiry, ql.Period(2, ql.Days))
flavor = 'call'
strike = 3200
notional = 1e4
notional_ccy = 'USD'
trade_direction = 'long'

inst = fx_vanilla.FxVanilla(d_ccy, f_ccy, calendar, expiry, payment_date, flavor, strike,
                            notional, notional_ccy, trade_direction)

In [5]:
inst.npv(today, fx_fwd_crv, d_crv, vol_surface, daycount=ql.Actual365Fixed())

np.float64(664.8243657925801)

In [6]:
inst.delta(today, fx_fwd_crv, d_crv, vol_surface, daycount=ql.Actual365Fixed())

np.float64(2.435173720414241)

In [7]:
inst.gamma(today, fx_fwd_crv, d_crv, vol_surface, daycount=ql.Actual365Fixed())

np.float64(0.004069988790433854)

In [8]:
inst.vega(today, fx_fwd_crv, d_crv, vol_surface, daycount=ql.Actual365Fixed())

np.float64(15.719033583592307)

In [9]:
inst.theta(today, fx_fwd_crv, d_crv, vol_surface, daycount=ql.Actual365Fixed())

np.float64(-1.1660791819493852)

In [10]:
inst.rho(today, fx_fwd_crv, d_crv, vol_surface, daycount=ql.Actual365Fixed()) * 100

np.float64(20.06832491191517)

In [11]:
inst.rho(today, fx_fwd_crv, d_crv, vol_surface, daycount=ql.Actual365Fixed(), tweak_type="market") * 100

np.float64(20.121665094569607)

In [12]:
inst.phi(today, fx_fwd_crv, d_crv, vol_surface, daycount=ql.Actual365Fixed(), tweak_type="pillar_rate") * 100

np.float64(-21.853332838122697)

In [13]:
inst.phi(today, fx_fwd_crv, d_crv, vol_surface, daycount=ql.Actual365Fixed(), tweak_type="market") * 100

np.float64(-21.911663340222276)

In [14]:
from nemesis.products.commodity.pm_forward_curve import PMForwardCurve
from nemesis.products.fx.fx_implied_curve import FXImpliedAssetCurve
from nemesis.products.fx.fx_vol_surface import FXVolSurface
from nemesis.utils import *
from nemesis.products.rates import *
from nemesis.market.curves import InterpTypes
from nemesis.utils.calendar import JointCalendar

####################################################################
#  NEMESIS ALPHA Version 0.1.0 - This build: 24 Jan 2025 at 10:42 #
####################################################################



In [15]:
value_dt = Date(11,8,2025)
spot_pm_rate = spot
currency_pair = "XAUUSD"

pm_forward_curve = PMForwardCurve(value_dt, spot_pm_rate, pm_fwd_data, "rate", currency_pair, JointCalendar([CalendarTypes.UNITED_KINGDOM, CalendarTypes.UNITED_STATES]), DayCountTypes.ACT_365F, InterpTypes.FLAT_FWD_RATES)
domestic_curve = QLCurve(value_dt, d_crv, dc_type=DayCountTypes.ACT_360, interp_type=InterpTypes.LINEAR_ZERO_RATES)
foreign_implied_curve = FXImpliedAssetCurve(value_dt, domestic_curve, pm_forward_curve, JointCalendar([CalendarTypes.UNITED_KINGDOM, CalendarTypes.UNITED_STATES]), DayCountTypes.ACT_365F, InterpTypes.LINEAR_ZERO_RATES)
pm_vol_surface = FXVolSurface(value_dt, vol_data, spot_pm_rate, currency_pair, pm_forward_curve, foreign_implied_curve, cal_type=JointCalendar([CalendarTypes.UNITED_KINGDOM, CalendarTypes.UNITED_STATES]), dc_type=DayCountTypes.ACT_365F)

In [16]:
from nemesis.products.fx.fx_vanilla_option import FXVanillaOption

In [17]:
expiry_dt = Date(13,11,2025)
strike_pm_rate = 3200
notional = 1e4
prem_currency = "USD"

o = FXVanillaOption(expiry_dt, strike_pm_rate, currency_pair, OptionTypes.EUROPEAN_CALL, notional, prem_currency, JointCalendar([CalendarTypes.CHINA, CalendarTypes.UNITED_STATES]), spot_days=2)

In [18]:
o.value(value_dt, pm_forward_curve, domestic_curve, pm_vol_surface, DayCountTypes.ACT_365F)

{'value': np.float64(212.74379705362614),
 'cash_dom': np.float64(664.8243657925817),
 'cash_for': np.float64(0.19890806996011265),
 'pips_dom': np.float64(212.74379705362614),
 'pips_for': np.float64(1.9890806996011263e-05),
 'pct_dom': np.float64(0.06648243657925818),
 'pct_for': np.float64(0.06365058238723605),
 'not_dom': 10000.0,
 'not_for': 3.125,
 'ccy_dom': 'USD',
 'ccy_for': 'XAU'}

In [19]:
o.delta(value_dt, pm_forward_curve, domestic_curve, pm_vol_surface, DayCountTypes.ACT_365F) * notional / strike_pm_rate

np.float64(2.43517371378843)

In [20]:
o.gamma(value_dt, pm_forward_curve, domestic_curve, pm_vol_surface, DayCountTypes.ACT_365F) * notional / strike_pm_rate

np.float64(0.0033661962106634746)

In [21]:
o.vega(value_dt, pm_forward_curve, domestic_curve, pm_vol_surface, DayCountTypes.ACT_365F) * notional / strike_pm_rate

np.float64(15.719033583592346)

In [22]:
o.theta(value_dt, pm_forward_curve, domestic_curve, pm_vol_surface, DayCountTypes.ACT_365F) * notional / strike_pm_rate

np.float64(-1.166079181949442)

In [23]:
o.rho(value_dt, pm_forward_curve, domestic_curve, pm_vol_surface, DayCountTypes.ACT_365F, bump_type="pillar") * notional / strike_pm_rate

np.float64(0.20068324911912683)

In [24]:
o.rho(value_dt, pm_forward_curve, domestic_curve, pm_vol_surface, DayCountTypes.ACT_365F, bump_type="market") * notional / strike_pm_rate

np.float64(0.201216650945657)

In [25]:
o.phi(value_dt, pm_forward_curve, domestic_curve, pm_vol_surface, DayCountTypes.ACT_365F, bump_type="pillar") * notional / strike_pm_rate

np.float64(0.21853332838124295)

In [26]:
o.phi(value_dt, pm_forward_curve, domestic_curve, pm_vol_surface, DayCountTypes.ACT_365F, bump_type="market") * notional / strike_pm_rate

np.float64(0.21911663340228316)